In [ ]:
#1.4.2. 斯坦福大学自然语言推理 (SNLI) 数据集¶
#斯坦福大学自然语言推理 (SNLI) 语料库是超过 50 万个标记为英语句子对 [Bowman et al., 2015] 的集合。
#我们将提取的 SNLI 数据集下载并存储在路径 ../data/snli_1.0 中。
!pip install d2l
import os
import re
import torch
from torch import nn
from d2l import torch as d2l

d2l.DATA_HUB['SNLI'] = (
    'https://nlp.stanford.edu/projects/snli/snli_1.0.zip',
    '9fcde07509c7e87ec61c640c1b2753d9041758e4')

data_dir = d2l.download_extract('SNLI')

In [ ]:
#1.4.2.1. 阅读数据集
#原始 SNLI 数据集包含的信息比我们实验中真正需要的信息丰富得多。
#因此，我们定义了一个函数 read_snli 来仅提取部分数据集，然后返回前提、假设及其标签的列表。
def read_snli(data_dir, is_train):
    """Read the SNLI dataset into premises, hypotheses, and labels."""
    def extract_text(s):
        # Remove information that will not be used by us
        s = re.sub('\\(', '', s)
        s = re.sub('\\)', '', s)
        # Substitute two or more consecutive whitespace with space
        s = re.sub('\\s{2,}', ' ', s)
        return s.strip()
    label_set = {'entailment': 0, 'contradiction': 1, 'neutral': 2}
    file_name = os.path.join(data_dir, 'snli_1.0_train.txt'
                             if is_train else 'snli_1.0_test.txt')
    with open(file_name, 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]
    premises = [extract_text(row[1]) for row in rows if row[0] in label_set]
    hypotheses = [extract_text(row[2]) for row in rows if row[0] in label_set]
    labels = [label_set[row[0]] for row in rows if row[0] in label_set]
    return premises, hypotheses, labels

In [ ]:
#现在让我们打印前 3 对前提和假设，以及它们的标签（“0”、“1” 和 “2” 分别对应于 “包容”、“矛盾” 和 “中性”）。
train_data = read_snli(data_dir, is_train=True)
for x0, x1, y in zip(train_data[0][:3], train_data[1][:3], train_data[2][:3]):
    print('premise:', x0)
    print('hypothesis:', x1)
    print('label:', y)

In [1]:
#该训练套装有大约 550,000 对，测试套装有约 10000 对。
#以下表明，在培训套装和测试套装中，“包容”、“矛盾” 和 “中性” 三个标签都是平衡的。
test_data = read_snli(data_dir, is_train=False)
for data in [train_data, test_data]:
    print([[row for row in data[2]].count(i) for i in range(3)])

In [ ]:
#1.4.2.2. 定义用于加载数据集的类
class SNLIDataset(torch.utils.data.Dataset):
    """A customized dataset to load the SNLI dataset."""
    def __init__(self, dataset, num_steps, vocab=None):
        self.num_steps = num_steps
        all_premise_tokens = d2l.tokenize(dataset[0])
        all_hypothesis_tokens = d2l.tokenize(dataset[1])
        if vocab is None:
            self.vocab = d2l.Vocab(all_premise_tokens + all_hypothesis_tokens,
                                   min_freq=5, reserved_tokens=['<pad>'])
        else:
            self.vocab = vocab
        self.premises = self._pad(all_premise_tokens)
        self.hypotheses = self._pad(all_hypothesis_tokens)
        self.labels = torch.tensor(dataset[2])
        print('read ' + str(len(self.premises)) + ' examples')

    def _pad(self, lines):
        return torch.tensor([d2l.truncate_pad(
            self.vocab[line], self.num_steps, self.vocab['<pad>'])
                         for line in lines])

    def __getitem__(self, idx):
        return (self.premises[idx], self.hypotheses[idx]), self.labels[idx]

    def __len__(self):
        return len(self.premises)

In [ ]:
#1.4.2.3. 把所有东西放在一起¶
#现在我们可以调用 read_snli 函数和 SNLIDataset 类来下载 SNLI 数据集，
#并返回 DataLoader 实例用于训练和测试集合，以及训练集的词汇表。值得注意的是，我们必须使用从训练集中构建的词汇和测试集合的词汇。因此，在训练集中训练的模型将不知道测试集中的任何新令牌。
def load_data_snli(batch_size, num_steps=50):
    """Download the SNLI dataset and return data iterators and vocabulary."""
    num_workers = d2l.get_dataloader_workers()
    data_dir = d2l.download_extract('SNLI')
    train_data = read_snli(data_dir, True)
    test_data = read_snli(data_dir, False)
    train_set = SNLIDataset(train_data, num_steps)
    test_set = SNLIDataset(test_data, num_steps, train_set.vocab)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                             shuffle=True,
                                             num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(test_set, batch_size,
                                            shuffle=False,
                                            num_workers=num_workers)
    return train_iter, test_iter, train_set.vocab

In [ ]:
#在这里，我们将批处理大小设置为 128，序列长度为 50，然后调用 load_data_snli 函数来获取数据迭代器和词汇。
#然后我们打印词汇量大小。
train_iter, test_iter, vocab = load_data_snli(128, 50)
len(vocab)

In [ ]:
#现在我们打印第一个迷你手表的形状。与情绪分析相反，我们有两个输入 X[0] 和 X[1]，代表对前提和假设。
for X, Y in train_iter:
    print(X[0].shape)
    print(X[1].shape)
    print(Y.shape)
    break